In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
test = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

df.shape, test.shape

### Unindo os dataframes para que métodos de correção atuem sobre os dois conjunto de dados.

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

### Listando as variáveis seus tipos de dados e ocorrencias de não nulos

In [ ]:
# Listando as variáveis e seus tipos
df_all.info()

### Listando as variáveis categóricas do conjunto de dados

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

## **Análise Descritiva dos dados**
### Para conhecer melhor os tipos de dados do dataset, optei por traçar um histograma para cada atributo numérico conforme mostrado abaixo.


In [ ]:
# Apresentação da distribuição de frequencia das variáveis contínuas
colunas = [c for c in df_all.columns if c not in ['Unnamed: 0','id',]]
df_all[colunas].hist(bins=50, figsize=(20,15));


### Abaixo apresento um mapa de calor com a correlação entre as variáveis numéricas. Pelo gráfico, observamos uma correlação forte de 0.97 entre as variáveis **Departure Delay in Minutes** e **Arrival Delay in Minutes** o que é óbvio pois se há atraso na decolagem a probabilidade de atrasar o pouso é grande, embora em alguns casos o piloto até consegue compensar a diferença durante o voo. Outra correlação significativa está entre as variáveis **Ease of Online Booking** e **Inflight wifi service** que ficou em 0.71, como mostrado no gráfico abaixo. 


In [ ]:
plt.figure(figsize=(13,10))
sns.heatmap(df_all[colunas].corr(),vmin=-1,vmax=1,cmap='Blues',annot=True);


### Verificando o resultado gráfico da variável **Satisfaction**

In [ ]:
df_all.describe().round(2)

In [ ]:

df_all['satisfaction'].value_counts().plot.bar();

In [ ]:
# Olhando a coluna satisfaction
df_all['satisfaction'].value_counts()

### Analisando as variáveis, identifiquei que somente a variável **'Arrivel Delay in Minutes'** apresenta valores nulos, sendo um total de 393 observações nulas, conforme apresentado abaixo|

In [ ]:
# Calculando a quantidade de valores nulos para a variável 'Arrival Delay in Minutes'
df_all['Arrival Delay in Minutes'].isna().sum()

### Para não excluir os registros nulos, optei por subtituir os valores nulos pela moda dos valores da variável 'Arrival Delay in Minutes'

In [ ]:
df_all['Arrival Delay in Minutes'].fillna(df_all['Arrival Delay in Minutes'].mode()[0], inplace=True)

In [ ]:
# Listando as variáveis e seus tipos, agora não há mais valores nulos.
df_all.info()

In [ ]:
# Gerando uma cópia do DataFrame
df_all2 = df_all.copy()

### Como as variáveis **Gender**, **Customer Type**, **Type of Travel** e **Class** são categóricas, criarei variáveis dummy que são variáveis binárias (0 ou 1) criadas para representar uma variável com duas ou mais categorias.

In [ ]:
# Criando variáveis dummy para as variáveis categóticas 'Gender', 'Customer Type', 'Type of Travel' e  'Class'
df_all = pd.get_dummies(df_all, columns=['Gender','Customer Type','Type of Travel','Class'])

In [ ]:
#Verificando os dados após criação das Dummies
df_all.head().T

In [ ]:
#Verificando os tipos de dados após criação das Dummies
df_all.info()

In [ ]:
#Definindo as colunas de entrada
feats = [c for c in df_all.columns if c not in ['Unnamed: 0','id']]
feats

In [ ]:

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
treino, teste = train_test_split(df_all[feats], test_size=0.20, random_state=42)

# Separando treino e validação
treino, validacao = train_test_split(treino, test_size=0.20, random_state=42)

treino.shape, validacao.shape, teste.shape


In [ ]:
#Definindo as colunas de entrada
feats = [c for c in df_all.columns if c not in ['Unnamed: 0','id','satisfaction']]
feats

In [ ]:
# Importando o modelo
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Instanciar o modelo
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

In [ ]:
# Treinar o modelo
rf.fit(treino[feats],treino['satisfaction'])

In [ ]:
# Prevendo os dados de Validação
preds_val = rf.predict(validacao[feats])

preds_val

In [ ]:
# Avaliando o desempenho do modelo
# Importanto a métrica
from sklearn.metrics import accuracy_score


In [ ]:
# Acurácia das previsões de validação
accuracy_score(validacao['satisfaction'], preds_val)

In [ ]:
# Medindo a acurácia nos dados de teste
preds_teste = rf.predict(teste[feats])

accuracy_score(teste['satisfaction'], preds_teste)


In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
fig=plt.figure(figsize=(10,15))
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh();

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(validacao['satisfaction'], preds_val);

In [ ]:
# Matriz de Confusão - Dados de teste
skplt.metrics.plot_confusion_matrix(teste['satisfaction'], preds_teste);

Professor Marcos, segui o roteiro que da aula 04 para elaborar este trabalho.